# Import datasets

In [1]:
import pandas as pd

In [2]:
popularity = pd.read_csv("steamcharts.csv")
appdata = pd.read_csv("steam_app_data.csv", dtype={10: "string"})
headers = pd.read_csv("id_name.csv")

# Keep only rows that we have player data for

**Part of the cleaning process, does nothing the second time around**

In [3]:
usable_ids = popularity['steam_appid'].unique()

In [4]:
appdata = appdata[appdata['steam_appid'].isin(usable_ids)]
headers = headers[headers['appid'].isin(usable_ids)]

## Write the changes back to file - save space

In [5]:
appdata.to_csv('steam_app_data.csv', index=False)
headers.to_csv('id_name.csv', index=False)

In [6]:
appdata.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'controller_support', 'dlc', 'detailed_description', 'about_the_game',
       'short_description', 'fullgame', 'supported_languages', 'header_image',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'legal_notice', 'drm_notice', 'ext_user_account_notice', 'developers',
       'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
       'platforms', 'metacritic', 'reviews', 'categories', 'genres',
       'screenshots', 'movies', 'recommendations', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors'],
      dtype='object')

In [7]:
popularity.columns

Index(['month', 'avg_players', 'gain', 'gain_percent', 'peak_players', 'name',
       'steam_appid'],
      dtype='object')

In [8]:
#Removing columns that will not be of use
appdata = appdata.drop(['name','dlc', 'detailed_description', 'about_the_game','short_description',
                        'fullgame','header_image','website', 'pc_requirements', 'mac_requirements','linux_requirements','legal_notice',
                        'drm_notice', 'ext_user_account_notice', 'demos', 'price_overview',
                        'packages', 'package_groups', 'reviews','screenshots','movies', 'achievements', 'controller_support', 'metacritic',
                        'support_info', 'background','content_descriptors', 'recommendations'], axis=1)
popularity = popularity.drop(['name', 'gain'],  axis=1)

In [11]:
appdata.columns

Index(['type', 'steam_appid', 'required_age', 'is_free', 'supported_languages',
       'developers', 'publishers', 'platforms', 'categories', 'genres',
       'release_date'],
      dtype='object')

In [12]:
#Changing data types
popularity['month'] = pd.to_datetime(popularity['month'], format='%b-%y')
#In case it interprets 25 as 1925 and not 2025
popularity.loc[popularity['month'].dt.year < 2000, 'month'] += pd.DateOffset(years=100)

## Merging datasets